<a href="https://colab.research.google.com/github/enti1701aaa/Final-Team1/blob/main/Finalproject_1%ED%8C%80%EC%95%88%EC%A7%80%EC%98%81_%EC%84%B8%EC%9D%BC%EB%8D%B0%EC%9D%B4%ED%84%B0API%ED%98%B8%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/weighted_score_above_08.csv')

/tmp/ipython-input-1023940785.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/weighted_score_above_08.csv')


#공통 전처리

In [ ]:
# 타임스탬프를 날짜데이터로 변환
df[['timestamp_created', 'timestamp_updated','author_last_played']] = df[['timestamp_created', 'timestamp_updated','author_last_played']].apply(
    pd.to_datetime, unit='s'
)

In [ ]:
# 5년 이내 데이터 사용

# timestamp_created 즉 리뷰 작성일자
# 1. 가장 최신 날짜 구하기
latest_date = df['timestamp_created'].max()

# 2. 5년 전 cutoff 계산
cutoff_date = latest_date - pd.DateOffset(years=5)

# 3. 필터링 (5년 이내 데이터만 남김)
df= df[df['timestamp_created'] >= cutoff_date].copy()

In [ ]:
# 리뷰어 특성 전처리
df = df[(df['author_num_reviews'] != 0)
  & (df["author_num_games_owned"] != 0)
  & (df['author_playtime_forever'] != 0)
  & (df["author_playtime_at_review"] != 0)]

In [ ]:
# 이상치기준으로 일반/코어유저 구분

# 분석 대상 컬럼 정의
cols = [
    'author_num_games_owned',
    'author_num_reviews',
    'author_playtime_forever',
    'author_playtime_last_two_weeks',
    'author_playtime_at_review',
    'author_last_played'
]

numeric_cols = df[cols].apply(pd.to_numeric, errors='coerce')

# 분위수 계산
Q1 = numeric_cols.quantile(0.25)
Q3 = numeric_cols.quantile(0.75)
IQR = Q3 - Q1

# 이상치 경계 계산
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치 여부 태깅
df['is_outlier'] = ((numeric_cols < lower_bound) | (numeric_cols > upper_bound)).any(axis=1)

# 두 그룹 분리
df_general = df[df['is_outlier'] == False]   # 일반 유저군 (대세 트렌드용)
df_core    = df[df['is_outlier'] == True]    # 핵심/특수 유저군 (상위/하위 특이 패턴용)

# -----------------------
# 예시 분석: 분위수 기반 요약
# -----------------------

# 일반 유저 트렌드
general_stats = df_general[cols].describe(percentiles=[0.5, 0.75, 0.9, 0.95])

# 핵심 유저 특이 패턴
core_stats = df_core[cols].describe(percentiles=[0.5, 0.75, 0.9, 0.95])

print("📊 일반 유저 트렌드 요약:")
print(general_stats)

print("\n🔥 핵심 유저(이상치) 패턴 요약:")
print(core_stats)

📊 일반 유저 트렌드 요약:
       author_num_games_owned  author_num_reviews  author_playtime_forever  \
count           138601.000000       138601.000000            138601.000000   
mean               202.219262           19.168000              1805.426043   
min                  1.000000            1.000000                 1.000000   
50%                138.000000           12.000000               840.000000   
75%                274.000000           27.000000              2429.000000   
90%                476.000000           49.000000              5011.000000   
95%                625.000000           64.000000              6894.000000   
max                941.000000           92.000000             13357.000000   
std                189.090592           19.760682              2360.477106   

       author_playtime_last_two_weeks  author_playtime_at_review  \
count                        138601.0              138601.000000   
mean                              0.0                 884.667167   

#세일 이벤트 수집

###유효 리뷰 필터링
 - 프로모션으로 무료 획득한 유저 (received_for_free=True)
 - 외부 키(번들, 다른 플랫폼 등)로 등록한 유저(steam_purchase == 0)

In [ ]:
df_filtered = df[(df["received_for_free"] == 0) & (df["steam_purchase"] == 1)].copy()

print("원본 리뷰 수:", len(df))
print("필터링 후 리뷰 수:", len(df_filtered))

원본 리뷰 수: 221651
필터링 후 리뷰 수: 149105


###무료게임(F2P)제거
- Steam Storefront appdetail 사용

In [ ]:
import requests, time, math, os, json, pandas as pd
from tqdm import tqdm

REVIEW_CSV = "weighted_score_above_08.csv"   # 파일경로
CACHE_CSV  = "appid_isfree_cache.csv"        # 중간 캐시
OUT_META   = "appid_meta_final.csv"          # 최종 메타 결과
OUT_MERGED = "steam_review_filtered.csv"     # 무료/비본편 제거된 최종 리뷰

In [ ]:
# appid 고유 목록 추출

appids = (
    df_filtered['appid']
    .dropna()
    .astype(int)
    .drop_duplicates()
    .tolist()
)
print(f"Unique appids: {len(appids):,}")

Unique appids: 9,606


In [ ]:
# 캐시 파일 불러오기(실패시 리스크 축소)
need_cols = ['appid','is_free','type','ok','error']
if os.path.exists(CACHE_CSV):
    cache = pd.read_csv(CACHE_CSV)
    if 'appid' in cache:
        cache['appid'] = cache['appid'].astype(int, errors='ignore')
    # 누락 컬럼 보강
    for c in need_cols:
        if c not in cache.columns:
            cache[c] = None
    cache = cache[need_cols].drop_duplicates(subset=['appid'], keep='last')
else:
    cache = pd.DataFrame(columns=need_cols)

done_appids = set(cache['appid'].tolist())
todo_appids = [a for a in appids if a not in done_appids]
print(f"Cached: {len(done_appids):,} | To fetch: {len(todo_appids):,}")

# 단일 appid 조회 (F2P 여부만 확보)
def fetch_appdetails(appid, max_retries=3, cooldown=0.5):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}&cc=us&l=en"
    for attempt in range(1, max_retries+1):
        try:
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            js = r.json()
            rec = js.get(str(appid), {})
            if not rec or not rec.get('success'):
                return {"appid": appid, "is_free": None, "type": None, "ok": False, "error": "success_false_or_no_data"}
            data = rec.get('data', {})
            return {
                "appid": appid,
                "is_free": data.get("is_free", None),  # True / False / None
                "type": data.get("type", None),        # 사용하지 않지만 캐시 호환 위해 저장
                "ok": True,
                "error": None
            }
        except Exception as e:
            err = str(e)[:200]
            if attempt < max_retries:
                time.sleep(cooldown * attempt)  # 점점 늘어나는 대기
            else:
                return {"appid": appid, "is_free": None, "type": None, "ok": False, "error": err}

# 조회 루프 (레이트 제한 고려)
rows = []
for a in tqdm(todo_appids, desc="Fetching appdetails"):
    rows.append(fetch_appdetails(a))
    time.sleep(0.4)  # 속도 조절 (필요시 조정)

# 캐시에 합치고 저장
if rows:
    part = pd.DataFrame(rows)
    cache = pd.concat([cache, part], ignore_index=True)
    # 중복 appid가 있다면 마지막 결과만 남기기
    cache = cache.drop_duplicates(subset=['appid'], keep='last')
    cache.to_csv(CACHE_CSV, index=False)

Cached: 0 | To fetch: 9,606


Fetching appdetails: 100%|██████████| 9606/9606 [2:49:50<00:00,  1.06s/it]


In [ ]:
# 최종 정리본 저장
meta = cache[['appid','is_free','type','ok','error']].copy()
meta.to_csv(OUT_META, index=False)
print(f"Saved meta: {OUT_META} ({len(meta):,} rows)")

# 리뷰 DF와 merge
m = df_filtered.merge(meta[['appid','is_free']], on='appid', how='left')

# F2P 플래그 컬럼: True/False/None 유지
m['f2p'] = m['is_free']

# 요약 리포트
total = len(m)
cnt_true  = (m['f2p'] == True).sum()
cnt_false = (m['f2p'] == False).sum()
cnt_none  = m['f2p'].isna().sum()
print(f"Reviews: {total:,} | f2p=True: {cnt_true:,} | f2p=False: {cnt_false:,} | f2p=None: {cnt_none:,}")

# 저장
m.to_csv(OUT_MERGED, index=False)
print(f"Saved reviews with f2p flag: {OUT_MERGED}")

Saved meta: appid_meta_final.csv (9,606 rows)
Reviews: 149,105 | f2p=True: 2,826 | f2p=False: 104,267 | f2p=None: 42,012
Saved reviews with f2p flag: steam_review_filtered.csv


In [ ]:
# f2p=False 인 것만 남기기

f2p_norm = m['f2p'].map({True: True, False: False, 'True': True, 'False': False, 1: True, 0: False})
m_f2p_false = m[f2p_norm == False].copy()
print(len(m), "->", len(m_f2p_false))

m_f2p_false.to_csv("steam_review_f2p_false.csv", index=False)

/tmp/ipython-input-845961437.py:2: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  m=pd.read_csv("/content/steam_review_filtered.csv")


149105 -> 104267


###플랫폼 세일 히스토리 수집
- SteamDB Sales History 크롤링

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Steam Sale Date.html to Steam Sale Date.html


In [ ]:
html_file = next(iter(uploaded.keys()))
print("uploaded:", html_file)

uploaded: Steam Sale Date.html


In [ ]:
import re, pandas as pd
from bs4 import BeautifulSoup
from dateutil import parser as dparser

DATE_LINE_RE = re.compile(r"(\d{1,2}\s+[A-Za-z]+(?:\s+\d{4})?)\s+—\s+(\d{1,2}\s+[A-Za-z]+(?:\s+\d{4})?)")

def nearest_year_from_heading(tag):
    cur = tag
    while cur:
        cur = cur.find_previous(["h3","h2"])
        if cur:
            m = re.search(r"\b(19|20)\d{2}\b", cur.get_text(strip=True))
            if m: return int(m.group(0))
    return pd.Timestamp.utcnow().year

def parse_date_range(date_text, default_year):
    m = DATE_LINE_RE.search(date_text or "")
    if not m:
        parts = (date_text or "").split("—")
        if len(parts)==2:
            left, right = parts[0].strip(), parts[1].strip()
            mon = re.search(r"[A-Za-z]+", right)
            if mon: left = f"{left} {mon.group(0)}"
            m = DATE_LINE_RE.search(f"{left} — {right}")
    if not m: return None, None
    s_raw, e_raw = m.group(1), m.group(2)

    def ensure_year(s, y):
        return s if re.search(r"\b(19|20)\d{2}\b", s) else f"{s} {y}"

    s_txt = ensure_year(s_raw, default_year)

    # 종료 연도 보정(12월→다음해 1월 케이스)
    if not re.search(r"\b(19|20)\d{2}\b", e_raw):
        s_dt_tmp = dparser.parse(s_txt, dayfirst=True, fuzzy=True)
        e_txt_tmp = f"{e_raw} {default_year}"
        e_dt_tmp = dparser.parse(e_txt_tmp, dayfirst=True, fuzzy=True)
        e_txt = f"{e_raw} {default_year+1}" if e_dt_tmp.month < s_dt_tmp.month else e_txt_tmp
    else:
        e_txt = e_raw

    s_dt = pd.to_datetime(s_txt, utc=True, dayfirst=True, errors="coerce").normalize()
    e_dt = pd.to_datetime(e_txt, utc=True, dayfirst=True, errors="coerce").normalize()
    return s_dt, e_dt

def classify_event(name: str):
    n = name.lower()
    if "sale" in n:
        if any(k in n for k in ["summer","winter","autumn","spring"]): return "seasonal_sale"
        return "sale"
    if "fest" in n: return "fest_next" if "next fest" in n else "fest"
    return "other"

# HTML 읽고 파싱
with open(html_file, "r", encoding="utf-8", errors="ignore") as f:
    html = f.read()
soup = BeautifulSoup(html, "lxml")

BASE = "https://steamdb.info"
events = []
for h4 in soup.find_all("h4"):
    title = h4.get_text(" ", strip=True).replace("…","").strip()
    a = h4.find("a")
    link = a["href"] if (a and a.has_attr("href")) else None
    if link and link.startswith("/"): link = BASE + link

    # h4 인접 텍스트에서 날짜 라인 찾기
    date_text, steps, sib = None, 0, h4.next_sibling
    while sib and steps < 6 and date_text is None:
        txt = sib.get_text(" ", strip=True) if hasattr(sib, "get_text") else str(sib).strip()
        if "—" in txt and re.search(r"[A-Za-z]+", txt):
            if DATE_LINE_RE.search(txt) or re.search(r"\d{1,2}\s*—\s*\d{1,2}\s+[A-Za-z]+", txt):
                date_text = txt; break
        sib = sib.next_sibling; steps += 1

    year_ctx = nearest_year_from_heading(h4)
    s_dt, e_dt = parse_date_range(date_text, year_ctx)
    if s_dt is None or e_dt is None:
        continue

    events.append({
        "event_name": title,
        "source_url": link,
        "year_context": year_ctx,
        "date_text_raw": date_text,
        "start_utc": s_dt,
        "end_utc": e_dt,
        "event_type": classify_event(title),
    })

df = (pd.DataFrame(events)
        .drop_duplicates(subset=["event_name","start_utc","end_utc"])
        .sort_values(["start_utc","end_utc","event_name"], ascending=[False, False, True])
        .reset_index(drop=True))

df.to_csv("steamdb_valve_events_history.csv", index=False)
print("Saved:", "steamdb_valve_events_history.csv", "| rows:", len(df))
df.head(20)

Saved: steamdb_valve_events_history.csv | rows: 207


,event_name,source_url,year_context,date_text_raw,start_utc,end_utc,event_type
0,Winter Sale 2025,https://steamcommunity.com/groups/steamworks/a...,2025,18 December — 5 January 2026,2025-12-18 00:00:00+00:00,2026-01-05 00:00:00+00:00,seasonal_sale
1,Sports Fest,https://steamcommunity.com/groups/steamworks/a...,2025,8 — 15 December,2025-12-08 00:00:00+00:00,2025-12-15 00:00:00+00:00,fest
2,Animal Fest,https://steamcommunity.com/groups/steamworks/a...,2025,10 — 17 November,2025-11-10 00:00:00+00:00,2025-11-17 00:00:00+00:00,fest
3,Steam Scream Fest 2025,https://steamcommunity.com/groups/steamworks/a...,2025,27 October — 3 November,2025-10-27 00:00:00+00:00,2025-11-03 00:00:00+00:00,fest
4,Next Fest: October 2025,https://steamcommunity.com/groups/store_promos...,2025,13 — 20 October,2025-10-13 00:00:00+00:00,2025-10-20 00:00:00+00:00,fest_next
5,Autumn Sale 2025,https://steamcommunity.com/groups/steamworks/a...,2025,29 September — 6 October,2025-09-29 00:00:00+00:00,2025-10-06 00:00:00+00:00,seasonal_sale
6,Political Sim Fest,https://steamcommunity.com/groups/store_promos...,2025,8 — 15 September,2025-09-08 00:00:00+00:00,2025-09-15 00:00:00+00:00,fest
7,Third Person Shooter (TPS) Fest,https://steamcommunity.com/groups/store_promos...,2025,25 August — 1 September,2025-08-25 00:00:00+00:00,2025-09-01 00:00:00+00:00,fest
8,4X Fest,https://steamcommunity.com/groups/store_promos...,2025,11 — 18 August,2025-08-11 00:00:00+00:00,2025-08-18 00:00:00+00:00,fest
9,Racing Fest,https://steamcommunity.com/groups/store_promos...,2025,28 July — 4 August,2025-07-28 00:00:00+00:00,2025-08-04 00:00:00+00:00,fest


###게임별 할인/가격 히스토리 수집
- ITAD(IsThereAnyDeal) API
- 가격/세일 히스토리 전체 수집 사이트
- SHOP_ID = 61 -> Steam 가격 변동 지표만 사용
- 속도 향상을 위해 50개 단위 append 저장
- 중단시 재개 원활을 위해 캐시 저장

In [ ]:
import os, json, time, itertools
import pandas as pd
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# 설정
API_KEY   = "fe20eb4bb09ac4ef74150d90cc0e1e071158508e"
CSV_IN    = "/content/steam_review_f2p_false.csv"       # appid 포함 CSV
COUNTRY   = "US"
SINCE     = "2019-01-01T00:00:00Z"
SHOP_ID   = 61                                          # Steam
OUT_FULL  = "sales_steam.csv"
CACHE_FILE= "itad_cache.json"
DONE_FILE = "processed_appids.json"
SAVE_EVERY= 50
SLEEP_SEC = 0.5

# HTTP 세션
def make_session():
    s = requests.Session()
    retries = Retry(
        total=5, backoff_factor=0.6,
        status_forcelist=(429,500,502,503,504),
        allowed_methods=["GET","POST"]
    )
    s.headers.update({"User-Agent":"itad-batch/1.2"})
    s.mount("https://", HTTPAdapter(max_retries=retries))
    return s
session = make_session()

# 유틸 함수 작성
def chunked(it, n):
    it = iter(it)
    while True:
        batch = list(itertools.islice(it, n))
        if not batch: break
        yield batch

def load_json(path, default):
    if os.path.exists(path):
        try:
            return json.load(open(path, "r"))
        except:
            return default
    return default

def save_json(path, obj):
    tmp = path + ".tmp"
    with open(tmp, "w") as f:
        json.dump(obj, f)
    os.replace(tmp, path)

# ITAD API
def lookup_batch(appids):
    url = f"https://api.isthereanydeal.com/lookup/id/shop/{SHOP_ID}/v1"
    body = [f"app/{a}" for a in appids]
    r = session.post(url, params={"key": API_KEY}, json=body, timeout=30)
    r.raise_for_status()
    data = r.json()
    return {int(k.split("/")[1]): v for k,v in data.items()}

def fetch_history(itad_id, country=COUNTRY, since=SINCE):
    url = "https://api.isthereanydeal.com/games/history/v2"
    params = {"key": API_KEY, "id": itad_id, "country": country}
    if since: params["since"] = since
    r = session.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()
    if isinstance(data, dict) and "history" in data:
        data = data["history"]
    return data if isinstance(data, list) else []

# 파서 (가격 스케일 픽스)
def parse_history_item(item):
    shop = item.get("shop", {})
    if not isinstance(shop, dict) or shop.get("id") != SHOP_ID:
        return None

    deal = item.get("deal", {})
    if not isinstance(deal, dict):
        return None

    price_obj = deal.get("price")
    price_raw = None
    price_major = None
    currency = None

    if isinstance(price_obj, dict):
        amount = price_obj.get("amount")
        currency = price_obj.get("currency")
        price_raw = amount
        if isinstance(amount, int):
            price_major = amount / 100.0
        elif isinstance(amount, float):
            price_major = float(amount)

    cut = deal.get("cut")

    return {
        "timestamp": item.get("timestamp"),
        "price_raw": price_raw,
        "price": price_major,
        "currency": currency,
        "cut": cut,
        "shop_id": shop.get("id"),
        "shop": shop.get("name"),
    }

# 입력 appid
df_in = pd.read_csv(CSV_IN)
all_appids = pd.Index(df_in["appid"].dropna().astype(int).unique()).tolist()
print(f"총 appid: {len(all_appids)}")

# 캐시/재개
cache = load_json(CACHE_FILE, {})
done  = set(load_json(DONE_FILE, []))

if os.path.exists(OUT_FULL):
    try:
        prev = pd.read_csv(OUT_FULL, usecols=["appid"])
        done |= set(prev["appid"].astype(int).unique().tolist())
    except Exception as e:
        print("[WARN] 기존 sales_steam.csv 읽기 실패:", e)

todo_appids = [a for a in all_appids if a not in done]
print(f"처리 대상: {len(todo_appids)}개 / 이미 완료: {len(done)}개")

# appid -> itad_id 매핑 선조회
missing = [a for a in todo_appids if str(a) not in cache]
for batch in chunked(missing, 100):
    try:
        res = lookup_batch(batch)
        for a in batch:
            cache[str(a)] = res.get(a)
        save_json(CACHE_FILE, cache)
        time.sleep(SLEEP_SEC)
    except requests.HTTPError as e:
        print(f"[LOOKUP ERR] size={len(batch)} -> {e}")
        time.sleep(SLEEP_SEC*3)

# 히스토리 수집
buffer, processed = [], 0
total = len(todo_appids)

def append_csv(rows):
    df = pd.DataFrame.from_records(rows)
    header = not os.path.exists(OUT_FULL)
    df.to_csv(OUT_FULL, mode="a", index=False, header=header)

for i, appid in enumerate(todo_appids, 1):
    itad_id = cache.get(str(appid))
    if not itad_id:
        done.add(appid)
        if i % SAVE_EVERY == 0: save_json(DONE_FILE, sorted(list(done)))
        continue

    try:
        hist = fetch_history(itad_id)
        kept = 0
        for h in hist:
            parsed = parse_history_item(h)
            if parsed:
                parsed["appid"] = appid
                parsed["itad_id"] = itad_id
                buffer.append(parsed)
                kept += 1

        done.add(appid)
        processed += 1

        if i % 50 == 0 or i == total:
            print(f"[{i}/{total}] appid {appid} -> kept {kept} (버퍼 {len(buffer)})")

        if processed >= SAVE_EVERY or len(buffer) >= 5000:
            if buffer:
                append_csv(buffer); buffer.clear()
            save_json(DONE_FILE, sorted(list(done)))
            save_json(CACHE_FILE, cache)
            processed = 0

        time.sleep(SLEEP_SEC)

    except requests.HTTPError as e:
        print(f"[HIST ERR] appid {appid} ({itad_id}) -> {e}")
        time.sleep(SLEEP_SEC*4)
    except Exception as e:
        print(f"[ERR] appid {appid}: {e}")
        time.sleep(SLEEP_SEC*2)

# 남은 버퍼 flush
if buffer:
    append_csv(buffer)
save_json(DONE_FILE, sorted(list(done)))
save_json(CACHE_FILE, cache)
print(f"완료 ✅ (출력: {OUT_FULL})")

# 세일 구간 요약
print("세일 구간 요약 생성 중...")
df_all = pd.read_csv(OUT_FULL)
if not df_all.empty:
    dfx = df_all.copy()
    dfx["ts"] = pd.to_datetime(dfx["timestamp"], utc=True, errors="coerce")
    dfx = dfx.dropna(subset=["ts"]).sort_values(["appid","ts"])
    dfx["on_sale"] = dfx["cut"].fillna(0).astype(float) > 0
    dfx["boundary"] = dfx.groupby("appid")["on_sale"].apply(lambda s: s.ne(s.shift(1))).reset_index(drop=True)
    dfx["block"] = dfx.groupby("appid")["boundary"].cumsum()

    sale_blocks = []
    for (appid, block), g in dfx.groupby(["appid","block"]):
        if not g["on_sale"].iloc[0]:
            continue
        sale_blocks.append({
            "appid": appid,
            "sale_start_utc": g["ts"].iloc[0],
            "sale_end_utc": g["ts"].iloc[-1],
            "discount_max": g["cut"].max(),
            "n_points": len(g)
        })
    sale_df = pd.DataFrame(sale_blocks).sort_values(["appid","sale_start_utc"])
    sale_out = "sale_periods.csv"
    sale_df.to_csv(sale_out, index=False)
    print(f"세일 요약 저장 → {sale_out}  (rows={len(sale_df)})")


총 appid: 6653
처리 대상: 6653개 / 이미 완료: 0개
[50/6653] appid 1008230 -> kept 43 (버퍼 3499)
[100/6653] appid 1016360 -> kept 48 (버퍼 4256)
[150/6653] appid 1023870 -> kept 23 (버퍼 3565)
[200/6653] appid 1045520 -> kept 100 (버퍼 3647)
[250/6653] appid 1054990 -> kept 72 (버퍼 3541)
[300/6653] appid 1061100 -> kept 7 (버퍼 3361)
[350/6653] appid 1066900 -> kept 54 (버퍼 3271)
[400/6653] appid 1087760 -> kept 60 (버퍼 3457)
[450/6653] appid 1094930 -> kept 100 (버퍼 3517)
[500/6653] appid 1101140 -> kept 131 (버퍼 3006)
[550/6653] appid 110800 -> kept 108 (버퍼 3569)
[600/6653] appid 1116210 -> kept 112 (버퍼 3346)
[650/6653] appid 1137350 -> kept 65 (버퍼 3572)
[700/6653] appid 1144440 -> kept 22 (버퍼 3375)
[750/6653] appid 1150590 -> kept 27 (버퍼 3593)
[800/6653] appid 1157220 -> kept 57 (버퍼 3586)
[850/6653] appid 1175480 -> kept 37 (버퍼 3489)
[900/6653] appid 1183470 -> kept 105 (버퍼 3393)
[950/6653] appid 1192440 -> kept 31 (버퍼 3075)
[1000/6653] appid 1200910 -> kept 114 (버퍼 2934)
[1050/6653] appid 1222700 -> kept 85

###리뷰 × 세일/이벤트 조인
- reviews_with_sale_flags.csv
    - steam_review_f2p_false.csv의 리뷰(appid, timestamp_created)를 기준으로 sale_periods.csv (게임 자체 세일 구간) steamdb_valve_events_history.csv (스팀 주최 이벤트 구간) 매칭시킴
    - 세일/이벤트 여부 플래그만 포함

- reviews_with_sale_flags_full.csv
    - reviews_with_sale_flags.csv와 원본 리뷰 데이터 steam_review_f2p_false.csv를 appid + timestamp_created 기준으로 조인
    - 리뷰 정보 + 세일/이벤트 정보 통합 최종 분석 데이터셋
    - 중복키 12건 있었으나 전체의 0.01%도 되지않아 그냥 drop해 한쪽만 남김

In [ ]:
import pandas as pd
import numpy as np

# 경로 설정
REVIEWS_CSV   = "/content/steam_review_f2p_false.csv"
SALE_PERIODS  = "/content/sale_periods.csv"
VALVE_EVENTS  = "/content/steamdb_valve_events_history.csv"
OUT_CSV       = "/content/reviews_with_sale_flags.csv"

# 데이터 로드
df = pd.read_csv(REVIEWS_CSV)
sp = pd.read_csv(SALE_PERIODS)
ve = pd.read_csv(VALVE_EVENTS)

# 리뷰: 필수 컬럼 확인
assert "appid" in df.columns, "reviews CSV에 appid 컬럼이 필요합니다."
assert "timestamp_created" in df.columns, "reviews CSV에 timestamp_created 컬럼이 필요합니다."

# 이벤트: 사용자가 보여준 컬럼에 맞춰 리네임 (start_utc/end_utc → event_start_utc/event_end_utc)
rename_map = {}
if "start_utc" in ve.columns: rename_map["start_utc"] = "event_start_utc"
if "end_utc"   in ve.columns: rename_map["end_utc"]   = "event_end_utc"
ve = ve.rename(columns=rename_map)

# 세일 요약: 필수 컬럼 확인
for col in ["appid", "sale_start_utc", "sale_end_utc"]:
    assert col in sp.columns, f"sale_periods CSV에 {col} 컬럼이 필요합니다."

# 이벤트: 필수 컬럼 확인
for col in ["event_start_utc", "event_end_utc"]:
    assert col in ve.columns, f"valve_events CSV에 {col} 컬럼이 필요합니다."

# 시간 정규화(UTC)
df["ts"] = pd.to_datetime(df["timestamp_created"], utc=True, errors="coerce")
sp["sale_start_utc"]   = pd.to_datetime(sp["sale_start_utc"],   utc=True, errors="coerce")
sp["sale_end_utc"]     = pd.to_datetime(sp["sale_end_utc"],     utc=True, errors="coerce")
ve["event_start_utc"]  = pd.to_datetime(ve["event_start_utc"],  utc=True, errors="coerce")
ve["event_end_utc"]    = pd.to_datetime(ve["event_end_utc"],    utc=True, errors="coerce")

# 결측 제거 & 정렬
df = df.dropna(subset=["appid", "ts"]).sort_values(["appid", "ts"]).reset_index(drop=True)
sp = sp.dropna(subset=["appid", "sale_start_utc", "sale_end_utc"]).sort_values(["appid", "sale_start_utc"])
ve = ve.dropna(subset=["event_start_utc", "event_end_utc"]).sort_values("event_start_utc")

# 게임 자체 세일 매칭 (appid 그룹별)
def asof_join_game_sale(group_df):
    a = group_df.sort_values("ts")
    b = sp[sp["appid"] == group_df.name].sort_values("sale_start_utc")
    if b.empty:
        a["sale_start_utc"] = pd.NaT
        a["sale_end_utc"]   = pd.NaT
        a["discount_max"]   = np.nan
        return a
    merged = pd.merge_asof(
        a,
        b[["sale_start_utc", "sale_end_utc", "discount_max"]],
        left_on="ts", right_on="sale_start_utc",
        direction="backward",
    )
    return merged

df = df.groupby("appid", group_keys=False).apply(asof_join_game_sale)

# 포함 여부 플래그 (리뷰 시점이 sale_end 이하)
df["on_game_sale"] = (df["ts"] <= df["sale_end_utc"]).fillna(False)

# 스팀 주최 이벤트 매칭
df = df.sort_values("ts")
df = pd.merge_asof(
    df,
    ve[["event_start_utc", "event_end_utc", "event_name", "event_type"]] if "event_name" in ve.columns else ve[["event_start_utc", "event_end_utc"]],
    left_on="ts", right_on="event_start_utc",
    direction="backward",
)
df["on_valve_event"] = (df["ts"] <= df["event_end_utc"]).fillna(False)

# 컨텍스트 라벨
def _ctx(row):
    g, v = bool(row["on_game_sale"]), bool(row["on_valve_event"])
    if g and v: return "both"
    if g and not v: return "game_only"
    if (not g) and v: return "event_only"
    return "none"

df["sale_context"] = df.apply(_ctx, axis=1)

# 저장
keep_cols = [
    "appid", "timestamp_created", "ts",
    "on_game_sale", "discount_max", "sale_start_utc", "sale_end_utc",
    "on_valve_event", "event_start_utc", "event_end_utc", "sale_context"
]
# 이벤트 메타가 있으면 포함
for meta_col in ["event_name", "event_type"]:
    if meta_col in df.columns:
        keep_cols.append(meta_col)

keep_cols = [c for c in keep_cols if c in df.columns]
df_out = df[keep_cols].copy()
df_out.to_csv(OUT_CSV, index=False)

# 요약 출력
print("Saved ->", OUT_CSV)
print(df_out["sale_context"].value_counts(dropna=False).to_string())


/tmp/ipython-input-4017389642.py:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("appid", group_keys=False).apply(asof_join_game_sale)


Saved -> /content/reviews_with_sale_flags.csv
sale_context
none          68848
event_only    34970
both            226
game_only       223


In [ ]:
import pandas as pd

# 경로
REVIEWS_CSV = "/content/steam_review_f2p_false.csv"
FLAGS_CSV   = "/content/reviews_with_sale_flags.csv"
OUT_CSV     = "/content/reviews_with_sale_flags_full.csv"

# 데이터 로드
df_reviews = pd.read_csv(REVIEWS_CSV)
df_flags   = pd.read_csv(FLAGS_CSV)

# datetime 변환
df_reviews["timestamp_created"] = pd.to_datetime(
    df_reviews["timestamp_created"], utc=True, errors="coerce"
)
df_flags["timestamp_created"] = pd.to_datetime(
    df_flags["timestamp_created"], utc=True, errors="coerce"
)

# 병합
df_merged = pd.merge(
    df_reviews,
    df_flags,
    on=["appid", "timestamp_created"],
    how="inner"
)

# 중복 체크
dupe_mask = df_merged.duplicated(subset=["appid","timestamp_created"], keep=False)
n_dupes = dupe_mask.sum()

if n_dupes > 0:
    print(f"[경고] 중복된 매칭 {n_dupes}건 발견 (appid+timestamp_created 기준).")
    # 중복된 행 미리보기
    print(df_merged.loc[dupe_mask, ["appid","timestamp_created"]].head(10))

    # 필요하면 여기서 drop
    df_merged = df_merged.drop_duplicates(subset=["appid","timestamp_created"], keep="first")
    print(f"→ 중복 제거 후 행 수: {len(df_merged)}")
else:
    print("중복 없음 ✅")

# 결과 확인
print("최종 데이터 크기:", df_merged.shape)
print(df_merged.head())

# 저장
df_merged.to_csv(OUT_CSV, index=False)
print(f"저장 완료 → {OUT_CSV}")


[경고] 중복된 매칭 12건 발견 (appid+timestamp_created 기준).
         appid         timestamp_created
7230   1091500 2020-12-10 00:51:06+00:00
7231   1091500 2020-12-10 00:51:06+00:00
7232   1091500 2020-12-10 00:51:06+00:00
7233   1091500 2020-12-10 00:51:06+00:00
24023  1277510 2021-01-28 02:48:07+00:00
24024  1277510 2021-01-28 02:48:07+00:00
24025  1277510 2021-01-28 02:48:07+00:00
24026  1277510 2021-01-28 02:48:07+00:00
46664  2096610 2022-11-17 19:14:42+00:00
46665  2096610 2022-11-17 19:14:42+00:00
→ 중복 제거 후 행 수: 104264
최종 데이터 크기: (104264, 38)
   recommendationid  appid            game     author_steamid  \
0         147449116     10  Counter-Strike  76561199183984450   
1         147374264     10  Counter-Strike  76561198099573060   
2         147357703     10  Counter-Strike  76561199080026894   
3         147284743     10  Counter-Strike  76561199137893460   
4         147263134     10  Counter-Strike  76561199013220131   

   author_num_games_owned  author_num_reviews  author_playtime_